<a href="https://colab.research.google.com/github/giuliocapecchi/IR_project/blob/main/IR_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch matplotlib nltk tqdm gdown

C'è da discutere di come caricare la collection sul notebook. Ho trovato due approcci:

## 1. Utilizzare drive.mount

Il problema di questo approccio è che la location del file può cambiare da utente ad utente e il file andrebbe condiviso con Tonellotto

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/MIRCV/IR_project/'MSMARCO Passages collection'

## 2. utilizzare gdown

Con questo modulo si possono scaricare files, quindi ho scaricato la collection e l'ho butttata sul mio drive (ci vogliono circa 30s/1 minuto di tempo)

In [4]:
import gdown

url = 'https://drive.google.com/uc?id=1_wXJjiwdgc9Kpt7o7atP8oWe-U4Z56hn'
gdown.download(url, 'test.tsv', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1_wXJjiwdgc9Kpt7o7atP8oWe-U4Z56hn
From (redirected): https://drive.google.com/uc?id=1_wXJjiwdgc9Kpt7o7atP8oWe-U4Z56hn&confirm=t&uuid=4f9da29b-beec-4d65-b098-f5a9cd1dd1cd
To: /content/test.tsv
100%|██████████| 3.06G/3.06G [00:31<00:00, 96.1MB/s]


'test.tsv'

In [5]:
import pandas as pd

df = pd.read_csv('test.tsv', sep='\t', header=None)

In [6]:
print(df.head(1))

   0                                                  1
0  0  The presence of communication amid scientific ...


In ogni caso dobbiamo:

- trovare una collection più piccola per cominciare (ok anche vaswani secondo me)
- realizzare una copia in locale di questo notebook, non si può tutte le volte aspettare di riscaricare 3GB di collection (in realtà anche si perchè ci mette 30s/1 minuto al download).


La copia locale forse la farei in un .py normale come suggerito da tonellotto (vedi repo mio github)

---

In [ ]:
import re
import string
import nltk

nltk.download("stopwords", quiet=True)

def preprocess(s):
    # lowercasing
    s = s.lower()
    # ampersand
    s = s.replace("&", " and ")
    # special chars
    s = s.translate(dict([(ord(x), ord(y)) for x, y in zip("‘’´“”–-", "'''\"\"--")]))
    # acronyms
    s = re.sub(r"\.(?!(\S[^. ])|\d)", "", s) # remove dots that are not part of an acronym
    # remove punctuation
    s = s.translate(str.maketrans(string.punctuation, " " * len(string.punctuation)))
    # strip whitespaces
    s = s.strip()
    while "  " in s:
        s = s.replace("  ", " ")
    # tokeniser
    s = s.split()
    # stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    s = [t for t in s if t not in stopwords]
    # stemming
    stemmer = nltk.stem.PorterStemmer().stem
    s = [stemmer(t) for t in s]
    return s

In [ ]:
import time

def profile(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        ms = (end - start) * 1000
        print(f"{f.__name__} ({ms:.3f} ms)")
        return result
    return f_timer